In [ ]:
# https://medium.com/@mijanr/reading-and-plotting-historical-crypto-data-using-binance-api-with-python-c02a436c0307
import pandas as pd
import matplotlib.pyplot as plot

import datetime

useLocalOrRemoteData = False

# Define the start and end times for the data
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=365)

# Convert the times to Unix timestamps in milliseconds
start_timestamp = int(start_time.timestamp() * 1000)
end_timestamp = int(end_time.timestamp() * 1000)

In [ ]:
from binance.client import Client
import binance
import os

if not useLocalOrRemoteData:
    api_key = os.environ.get('somekey1')
    api_secret = os.environ.get('somekey2')

    client = Client(api_key, api_secret)
    clientData = client.get_account()
    balances = clientData["balances"] # this is the table we're interested in
    # print(balances)

In [ ]:
import json 

if useLocalOrRemoteData:
    # reading the data from the file 
    with open('account.json') as f: 
        data = f.read().replace('\'', '\"').replace("True", "\"True\"").replace("False", "\"False\"") 

    js = json.loads(data)
    balances = js["balances"]
    # print(balances)

In [ ]:
df = pd.DataFrame.from_dict(balances)
df.head()

# format and clean data
df["free"] = df["free"].transform(lambda x: float(x))
df["locked"]=df["locked"].transform(lambda x: float(x))
df = df[(df['free'] > 0) | (df['locked'] > 0)]
df.head()

df["price"]=0.
df["value"]=0.

In [ ]:
# compute data

for asset in df["asset"]:
    print("asset: ", asset+"USDT")
    # asset_price = client.get_symbol_ticker(symbol=asset+"USDT")
    asset_price = -1
    try:
        response = client.get_symbol_ticker(symbol=asset+"USDT")
        asset_price = float(response['price'])
        df.loc[df['asset'] == asset, 'price'] = asset_price
        value = df.loc[df['asset'] == asset, 'value']
        value = value + asset_price * df.loc[df['asset'] == asset, 'free']
        value = value + asset_price * df.loc[df['asset'] == asset, 'locked']
        df.loc[df['asset'] == asset, 'value'] = value
        print(df[df['asset'] == asset])
    except Exception as berr:
        print("Binance error: ", berr)

In [ ]:
# plot data

import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(
    header={ "values":list(df.columns),
                "fill_color":'paleturquoise',
                "align":'left'},
    cells={"values":df.transpose().values.tolist(),
               "fill_color":'lavender',
               "align":'left'})
])

fig.show()